# Librerias y dependencias

In [4]:
# Analisis y manipulacion de datos
import pandas as pd
import numpy as np

# Limpieza y normnalziacionb
import unicodedata
import re
import os

# Preprocesamiento

## Visitas a los puntos de informacion turistica

### Definicion de mapas de normalziacion

In [3]:
# City name mapping dictionary for common variations
city_mappings = {
    'BOGOTA': 'BOGOTA',
    'BOGOTA DC': 'BOGOTA',
    'BOGOTA D.C.': 'BOGOTA',
    'SANTA FE DE BOGOTA': 'BOGOTA',
    'MEDELLIN': 'MEDELLIN',
    'ANTIOQUIA': 'ANTIOQUIA',
    'VALLE DEL CAUCA': 'VALLE DEL CAUCA',
    'VALLE': 'VALLE DEL CAUCA',
    'CUNDINAMARCA': 'CUNDINAMARCA',
    'SANTANDER': 'SANTANDER',
    'NORTE DE SANTANDER': 'NORTE DE SANTANDER',
    'BOYACA': 'BOYACA',
    'VILLA DE LEYVA': 'VILLA DE LEYVA',
    'RAQUIRA': 'RAQUIRA',
    'MONIQUIRA': 'MONIQUIRA',
    'CHIQUINQUIRA': 'CHIQUINQUIRA',
    'SACHICA': 'SACHICA',
    'SUTAMARCHAN': 'SUTAMARCHAN',
    'TUNJA': 'TUNJA',
    'PAIPA': 'PAIPA',
    'DUITAMA': 'DUITAMA',
    'SOGAMOSO': 'SOGAMOSO',
    'NOBSA': 'NOBSA',
    'TIBASOSA': 'TIBASOSA',
    'AQUITANIA': 'AQUITANIA',
    'MONGUI': 'MONGUI',
    'IZA': 'IZA',
    'TOTA': 'TOTA',
    'CUITIVA': 'CUITIVA',
    'FIRAVITOBA': 'FIRAVITOBA',
    'COMBITA': 'COMBITA',
    'VENTAQUEMADA': 'VENTAQUEMADA',
    'SAMACA': 'SAMACA',
    'CUCAITA': 'CUCAITA',
    'MOTAVITA': 'MOTAVITA',
    'OICATA': 'OICATA',
    'TOCA': 'TOCA',
    'SORACA': 'SORACA',
    'SORA': 'SORA',
    'CHIVATA': 'CHIVATA',
    'SIACHOQUE': 'SIACHOQUE',
    'SANTA SOFIA': 'SANTA SOFIA',
    'NA': 'NA'
}

# Improved travel motivation mapping dictionary - consolidated from 37 to 8 main categories
travel_motivation_mappings = {
    # TURISMO CULTURAL Y PATRIMONIAL (Cultural and Heritage Tourism)
    'TURISMO CULTURAL': 'TURISMO CULTURAL Y PATRIMONIAL',
    'HISTORIA': 'TURISMO CULTURAL Y PATRIMONIAL',
    'RESENA HISTORICA': 'TURISMO CULTURAL Y PATRIMONIAL',
    'HISTORIA PATRIA': 'TURISMO CULTURAL Y PATRIMONIAL',
    'NOS GUSTA EL MONUMENTO IMPONENTE': 'TURISMO CULTURAL Y PATRIMONIAL',
    'CONOCIENDO': 'TURISMO CULTURAL Y PATRIMONIAL',
    'CONOCER HISTORIA': 'TURISMO CULTURAL Y PATRIMONIAL',
    'CONOCER': 'TURISMO CULTURAL Y PATRIMONIAL',
    
    # TURISMO DE NATURALEZA Y AVENTURA (Nature and Adventure Tourism)
    'TURISMO DE NATURALEZA': 'TURISMO DE NATURALEZA Y AVENTURA',
    'TURISMO DE AVENTURA': 'TURISMO DE NATURALEZA Y AVENTURA',
    'BICITURISMO': 'TURISMO DE NATURALEZA Y AVENTURA',
    'COMPETENCIA DE CICLISMO CAMPO VERDE': 'TURISMO DE NATURALEZA Y AVENTURA',
    
    # VACACIONES Y RECREACION (Vacation and Recreation)
    'VACACIONES RECREO Y OCIO': 'VACACIONES Y RECREACION',
    'TURISMO': 'VACACIONES Y RECREACION',
    'VACACIONAL': 'VACACIONES Y RECREACION',
    'TURISTICO': 'VACACIONES Y RECREACION',
    'TURISTICA': 'VACACIONES Y RECREACION',
    'RECREACION': 'VACACIONES Y RECREACION',
    'RECREATIVO': 'VACACIONES Y RECREACION',
    'VACACIONES': 'VACACIONES Y RECREACION',
    'DESCANSO': 'VACACIONES Y RECREACION',
    'OCIO': 'VACACIONES Y RECREACION',
    'PLACER': 'VACACIONES Y RECREACION',
    'ENTRETENIMIENTO': 'VACACIONES Y RECREACION',
    
    # TURISMO LOCAL Y SERVICIOS TURISTICOS (Local Tourism and Tourist Services)
    'TURISMO LOCAL Y DEPARTAMENTAL': 'TURISMO LOCAL Y SERVICIOS TURISTICOS',
    'ATRACTIVOS Y PRODUCTOS TURISTICOS': 'TURISMO LOCAL Y SERVICIOS TURISTICOS',
    'SERVICIO AL TURISTA': 'TURISMO LOCAL Y SERVICIOS TURISTICOS',
    'AGENTES DE VIAJE Y TURISMO': 'TURISMO LOCAL Y SERVICIOS TURISTICOS',
    'ALOJAMIENTO': 'TURISMO LOCAL Y SERVICIOS TURISTICOS',
    
    # NEGOCIOS Y TRABAJO (Business and Work)
    'NEGOCIOS Y MOTIVOS PROFESIONALES': 'NEGOCIOS Y TRABAJO',
    'NEGOCIOS': 'NEGOCIOS Y TRABAJO',
    'TRABAJO': 'NEGOCIOS Y TRABAJO',
    'LABORAL': 'NEGOCIOS Y TRABAJO',
    'EMPRESARIAL': 'NEGOCIOS Y TRABAJO',
    'COMERCIAL': 'NEGOCIOS Y TRABAJO',
    'PROFESIONAL': 'NEGOCIOS Y TRABAJO',
    'REUNION': 'NEGOCIOS Y TRABAJO',
    'REUNIONES': 'NEGOCIOS Y TRABAJO',
    'CONFERENCIA': 'NEGOCIOS Y TRABAJO',
    'COMPRAS': 'NEGOCIOS Y TRABAJO',
    
    # EDUCACION Y FORMACION (Education and Training)
    'EDUCACION Y FORMACION': 'EDUCACION Y FORMACION',
    'TURISMO EDUCATIVO': 'EDUCACION Y FORMACION',
    'ESTUDIO': 'EDUCACION Y FORMACION',
    'ESTUDIOS': 'EDUCACION Y FORMACION',
    'EDUCACION': 'EDUCACION Y FORMACION',
    'EDUCATIVO': 'EDUCACION Y FORMACION',
    'ACADEMICO': 'EDUCACION Y FORMACION',
    'UNIVERSIDAD': 'EDUCACION Y FORMACION',
    'CURSO': 'EDUCACION Y FORMACION',
    'FORMACION': 'EDUCACION Y FORMACION',
    'CAPACITACION': 'EDUCACION Y FORMACION',
    'INVESTIGACION': 'EDUCACION Y FORMACION',
    
    # SALUD Y BIENESTAR (Health and Wellness)
    'SALUD Y ATENCION MEDICA': 'SALUD Y BIENESTAR',
    'TURISMO DE BIENESTAR': 'SALUD Y BIENESTAR',
    'SERVICIOS DE BELLEZA': 'SALUD Y BIENESTAR',
    'SALUD': 'SALUD Y BIENESTAR',
    'MEDICO': 'SALUD Y BIENESTAR',
    'MEDICINA': 'SALUD Y BIENESTAR',
    'TRATAMIENTO': 'SALUD Y BIENESTAR',
    'HOSPITAL': 'SALUD Y BIENESTAR',
    'CLINICA': 'SALUD Y BIENESTAR',
    'TERAPIA': 'SALUD Y BIENESTAR',
    'REHABILITACION': 'SALUD Y BIENESTAR',
    
    # MOTIVOS PERSONALES Y FAMILIARES (Personal and Family Reasons)
    'VISITAS FAMILIARES Y AMIGOS': 'MOTIVOS PERSONALES Y FAMILIARES',
    'VISITA FAMILIAR': 'MOTIVOS PERSONALES Y FAMILIARES',
    'FAMILIA': 'MOTIVOS PERSONALES Y FAMILIARES',
    'FAMILIAR': 'MOTIVOS PERSONALES Y FAMILIARES',
    'PERSONAL': 'MOTIVOS PERSONALES Y FAMILIARES',
    'VISITA': 'MOTIVOS PERSONALES Y FAMILIARES',
    'VISITAR': 'MOTIVOS PERSONALES Y FAMILIARES',
    'AMIGOS': 'MOTIVOS PERSONALES Y FAMILIARES',
    'PARIENTES': 'MOTIVOS PERSONALES Y FAMILIARES',
    'MATRIMONIO': 'MOTIVOS PERSONALES Y FAMILIARES',
    'BODA': 'MOTIVOS PERSONALES Y FAMILIARES',
    'FUNERAL': 'MOTIVOS PERSONALES Y FAMILIARES',
    'CELEBRACION': 'MOTIVOS PERSONALES Y FAMILIARES',
    
    # TURISMO RELIGIOSO (Religious Tourism)
    'RELIGIONPEREGRINACION': 'TURISMO RELIGIOSO',
    'TURISMO RELIGIOSO': 'TURISMO RELIGIOSO',
    'RELIGIOSO': 'TURISMO RELIGIOSO',
    'RELIGION': 'TURISMO RELIGIOSO',
    'RELIGIOSA': 'TURISMO RELIGIOSO',
    'PEREGRINACION': 'TURISMO RELIGIOSO',
    'ESPIRITUAL': 'TURISMO RELIGIOSO',
    'IGLESIA': 'TURISMO RELIGIOSO',
    'MISA': 'TURISMO RELIGIOSO',
    'CULTO': 'TURISMO RELIGIOSO',
    
    # GASTRONOMIA (Gastronomy) - separate category for food tourism
    'ALIMENTOS BEBIDAS Y GASTRONOMIA': 'GASTRONOMIA',
    
    # TRANSITO Y OTROS (Transit and Others)
    'TRANSITO': 'TRANSITO Y OTROS',
    'PASO': 'TRANSITO Y OTROS',
    'CONEXION': 'TRANSITO Y OTROS',
    'ESCALA': 'TRANSITO Y OTROS',
    'TRANSFERENCIA': 'TRANSITO Y OTROS',
    'OTRAS': 'TRANSITO Y OTROS',
    'OTRO': 'TRANSITO Y OTROS',
    'OTROS': 'TRANSITO Y OTROS',
    'OTRA': 'TRANSITO Y OTROS',
    'DIFERENTES': 'TRANSITO Y OTROS',
    'VARIOS': 'TRANSITO Y OTROS',
    'MULTIPLE': 'TRANSITO Y OTROS',
    'NS/NR': 'TRANSITO Y OTROS',
    'NO SABE': 'TRANSITO Y OTROS',
    'NO RESPONDE': 'TRANSITO Y OTROS',
    'SIN ESPECIFICAR': 'TRANSITO Y OTROS'
}

# Transportation mode mapping dictionary for variations consolidation
transportation_mappings = {
    # Private vehicle variations
    'AUTOMOVIL': 'VEHICULO PARTICULAR',
    'AUTO': 'VEHICULO PARTICULAR',
    'CARRO': 'VEHICULO PARTICULAR',
    'VEHICULO PROPIO': 'VEHICULO PARTICULAR',
    'VEHICULO PARTICULAR': 'VEHICULO PARTICULAR',
    'COCHE': 'VEHICULO PARTICULAR',
    'VEHICULO PRIVADO': 'VEHICULO PARTICULAR',
    'CAMIONETA': 'VEHICULO PARTICULAR',
    'CAMIONETA PROPIA': 'VEHICULO PARTICULAR',
    'PICK UP': 'VEHICULO PARTICULAR',
    
    # Bus variations
    'BUS': 'AUTOBUS',
    'AUTOBUS': 'AUTOBUS',
    'OMNIBUS': 'AUTOBUS',
    'COLECTIVO': 'AUTOBUS',
    'BUSETA': 'AUTOBUS',
    'MICROBUS': 'AUTOBUS',
    'BUS INTERMUNICIPAL': 'AUTOBUS',
    'BUS URBANO': 'AUTOBUS',
    'TRANSPORTE PUBLICO': 'AUTOBUS',
    'SERVICIO PUBLICO': 'AUTOBUS',
    
    # Motorcycle variations
    'MOTOCICLETA': 'MOTOCICLETA',
    'MOTO': 'MOTOCICLETA',
    'MOTOCICLO': 'MOTOCICLETA',
    'CICLOMOTOR': 'MOTOCICLETA',
    'SCOOTER': 'MOTOCICLETA',
    
    # Bicycle variations
    'BICICLETA': 'BICICLETA',
    'BICI': 'BICICLETA',
    'CICLA': 'BICICLETA',
    
    # Airplane variations
    'AVION': 'AVION',
    'AEREO': 'AVION',
    'VUELO': 'AVION',
    'TRANSPORTE AEREO': 'AVION',
    'LINEA AEREA': 'AVION',
    
    # Taxi/Ride sharing variations
    'TAXI': 'TAXI',
    'UBER': 'TAXI/PLATAFORMA',
    'DIDI': 'TAXI/PLATAFORMA',
    'CABIFY': 'TAXI/PLATAFORMA',
    'BEAT': 'TAXI/PLATAFORMA',
    'PLATAFORMA DIGITAL': 'TAXI/PLATAFORMA',
    'APLICACION': 'TAXI/PLATAFORMA',
    
    # Train variations
    'TREN': 'TREN',
    'FERROCARRIL': 'TREN',
    'METRO': 'TREN',
    'TRANSMILENIO': 'TRANSPORTE MASIVO',
    'BRT': 'TRANSPORTE MASIVO',
    'SISTEMA INTEGRADO': 'TRANSPORTE MASIVO',
    
    # Walking variations
    'CAMINANDO': 'A PIE',
    'PIE': 'A PIE',
    'A PIE': 'A PIE',
    'PEATONAL': 'A PIE',
    'CAMINATA': 'A PIE',
    
    # Boat/Water transport
    'BARCO': 'TRANSPORTE ACUATICO',
    'LANCHA': 'TRANSPORTE ACUATICO',
    'EMBARCACION': 'TRANSPORTE ACUATICO',
    'FLUVIAL': 'TRANSPORTE ACUATICO',
    'ACUATICO': 'TRANSPORTE ACUATICO',
    
    # Rental/Tour vehicles
    'VEHICULO ALQUILADO': 'VEHICULO ALQUILADO',
    'ALQUILER': 'VEHICULO ALQUILADO',
    'RENTAL': 'VEHICULO ALQUILADO',
    'RENT A CAR': 'VEHICULO ALQUILADO',
    'TOUR': 'TOUR/EXCURSION',
    'EXCURSION': 'TOUR/EXCURSION',
    'PAQUETE TURISTICO': 'TOUR/EXCURSION',
    'AGENCIA': 'TOUR/EXCURSION',
    
    # Multiple/Combined transport
    'MIXTO': 'TRANSPORTE MIXTO',
    'COMBINADO': 'TRANSPORTE MIXTO',
    'VARIOS': 'TRANSPORTE MIXTO',
    'MULTIPLE': 'TRANSPORTE MIXTO',
    
    # Other/Unspecified
    'OTRO': 'OTROS',
    'OTROS': 'OTROS',
    'OTRA': 'OTROS',
    'DIFERENTE': 'OTROS',
    'NS/NR': 'NO ESPECIFICA',
    'NO SABE': 'NO ESPECIFICA',
    'NO RESPONDE': 'NO ESPECIFICA',
    'SIN ESPECIFICAR': 'NO ESPECIFICA',
    'NO APLICA': 'NO ESPECIFICA'
}

### Funciones de normalziacion

In [4]:
def normalize_text(text):
    """
    Normalize text by removing accents, converting to uppercase, and cleaning special characters
    """
    if pd.isna(text) or text == 'NA':
        return text
    
    # Convert to string and strip whitespace
    text = str(text).strip()
    
    # Remove accents and normalize unicode characters
    text = unicodedata.normalize('NFD', text)
    text = ''.join(char for char in text if unicodedata.category(char) != 'Mn')
    
    # Convert to uppercase for consistency
    text = text.upper()
    
    # Remove extra spaces and special characters (except essential ones)
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single space
    text = re.sub(r'[^\w\s-]', '', text)  # Keep only alphanumeric, spaces, and hyphens
    
    return text.strip()

def apply_city_mapping(normalized_city):
    """Apply manual mapping for known city variations"""
    if pd.isna(normalized_city) or normalized_city == 'NA':
        return normalized_city
    return city_mappings.get(normalized_city, normalized_city)

def apply_travel_motivation_mapping(normalized_motivation):
    """Apply manual mapping for known travel motivation variations"""
    if pd.isna(normalized_motivation) or normalized_motivation == 'NA':
        return normalized_motivation
    return travel_motivation_mappings.get(normalized_motivation, normalized_motivation)

def apply_transportation_mapping(normalized_transport):
    """Apply manual mapping for known transportation variations"""
    if pd.isna(normalized_transport) or normalized_transport == 'NA':
        return normalized_transport
    return transportation_mappings.get(normalized_transport, normalized_transport)

### Preprocesamiento

In [5]:
# Load dataset
df = pd.read_csv('./Datasets/Datos_de_las_Visitas_a_los_Puntos_de_Información_Turística_(PITS)_en_Boyacá_20250919.csv')

df['TIEMPO DE PERMANENCIA EN DIAS'] = df['TIEMPO DE PERMANENCIA EN DIAS'].astype('Int64')
df['CANTIDAD DE VIAJEROS'] = df['CANTIDAD DE VIAJEROS'].astype('Int64')

# Apply normalization to city columns
city_columns = ['CIUDAD DE PROCEDENCIA', 'CIUDAD DE DESTINO 1', 'CIUDAD DE DESTINO 2', 
                'CIUDAD DE DESTINO 3', 'CIUDAD DE DESTINO 4', 'CIUDAD DE DESTINO 5']

# Create normalized versions of city columns
for col in city_columns:
    df[f'{col}_NORMALIZED'] = df[col].apply(normalize_text)

# Apply manual mappings to normalized city columns
for col in city_columns:
    normalized_col = f'{col}_NORMALIZED'
    df[f'{col}_CLEAN'] = df[normalized_col].apply(apply_city_mapping)

# Apply normalization and mapping to travel motivation
df['MOTIVO_NORMALIZED'] = df['MOTIVO DEL VIAJE'].apply(normalize_text)
df['MOTIVO_CLEAN'] = df['MOTIVO_NORMALIZED'].apply(apply_travel_motivation_mapping)

# Apply normalization and mapping to transportation mode
df['TRANSPORTE_NORMALIZED'] = df['TIPO TRANSPORTE'].apply(normalize_text)
df['TRANSPORTE_CLEAN'] = df['TRANSPORTE_NORMALIZED'].apply(apply_transportation_mapping)

In [6]:
# Limpieza adicional de datos
# 1. Eliminar filas con NA en CIUDAD DE DESTINO 1
df = df.dropna(subset=['CIUDAD DE DESTINO 1'])

# 2. Remover columnas especificadas
columns_to_remove = ['TIPO ALOJAMIENTO', 'TIPO TRANSPORTE', 'PUNTO DE INFORMACIÓN', 'PRESUPUESTO', 'RESULTADO SATISFACCIÓN', 
                     'CIUDAD DE PROCEDENCIA', 'CIUDAD DE DESTINO 1', 'CIUDAD DE DESTINO 2', 'CIUDAD DE DESTINO 3', 
                     'CIUDAD DE DESTINO 4', 'CIUDAD DE DESTINO 5', 'CIUDAD DE PROCEDENCIA_NORMALIZED', 
                     'CIUDAD DE DESTINO 1_NORMALIZED', 'CIUDAD DE DESTINO 2_NORMALIZED','CIUDAD DE DESTINO 3_NORMALIZED', 
                     'CIUDAD DE DESTINO 4_NORMALIZED', 'CIUDAD DE DESTINO 5_NORMALIZED', 'MOTIVO_NORMALIZED', 'TRANSPORTE_NORMALIZED']
df = df.drop(columns=columns_to_remove, errors='ignore')

# 3. Filtrar CANTIDAD DE VIAJEROS entre 1 y 40
df = df[(df['CANTIDAD DE VIAJEROS'] >= 1) & (df['CANTIDAD DE VIAJEROS'] <= 40)]



In [7]:
# Renombrar automáticamente todas las columnas que terminen en "_CLEAN"
df.columns = [col.replace('_CLEAN', '') if col.endswith('_CLEAN') else col for col in df.columns]

# Reorganizar las columnas en el mismo orden que el dataset original
# Orden deseado basado en el dataset original (solo las columnas que quedan después del procesamiento)
desired_column_order = [
    'MES',
    'AÑO', 
    'PAIS DE PROCEDENCIA',
    'CIUDAD DE PROCEDENCIA',
    'CIUDAD DE DESTINO 1',
    'CIUDAD DE DESTINO 2', 
    'CIUDAD DE DESTINO 3',
    'CIUDAD DE DESTINO 4',
    'CIUDAD DE DESTINO 5',
    'SEXO',
    'MOTIVO DEL VIAJE',
    'TIEMPO DE PERMANENCIA EN DIAS',
    'CANTIDAD DE VIAJEROS',
    'TIPO TRANSPORTE'
]

# Verificar qué columnas existen realmente en el DataFrame y reordenar solo esas
existing_columns = [col for col in desired_column_order if col in df.columns]
other_columns = [col for col in df.columns if col not in desired_column_order]

# Reordenar: primero las columnas en el orden deseado, luego cualquier otra columna adicional
df = df[existing_columns + other_columns]


### Almacenamiento

In [8]:
# Guardar el dataset procesado
df.to_csv('./Datasets/Visitas_PIT_procesado.csv', index=False, na_rep='NA')

## Directyorio de municipios

### Preprocesamiento

In [47]:
# Load dataset
df = pd.read_csv('./Datasets/Directorio_de_municipios_y_alcaldías_-_DEPARTAMENTO_DE_BOYACÁ_20250924.csv')

# Extraer valores numéricos de la cadena
df[['LONGITUD', 'LATITUD']] = df['UBICACION (Fuente DANE)'].str.extract(r'POINT \((-?\d+\.\d+) (-?\d+\.\d+)\)')

# Convertir a float
df['LONGITUD'] = df['LONGITUD'].astype(float)
df['LATITUD'] = df['LATITUD'].astype(float)

columns_to_remove = ['CODIGO DANE MUNICIPIO', 'NIT ALCALDÍA', 'NOMBRE DEL ALCALDE', 'DIRECCIÓN', 'PAGINA WEB', 
                        'CORREOELECTRÓNICO', 'TELEFONO', 'ALTURA (msnm)', 'UBICACION (Fuente DANE)']
df = df.drop(columns=columns_to_remove, errors='ignore')

### Almacenamiento

In [48]:
# Guardar el dataset procesado
df.to_csv('./Datasets/Directorio_municipios_procesado.csv', index=False)

## Datos de entrenamiento modelo de prediccion

In [14]:
# Cargar dataset
df = pd.read_csv('./Datasets/Visitas_PIT_procesado.csv')

# Columnas de destino
cols_destinos = ['CIUDAD DE DESTINO 1','CIUDAD DE DESTINO 2',
                 'CIUDAD DE DESTINO 3','CIUDAD DE DESTINO 4',
                 'CIUDAD DE DESTINO 5']

# Pasar de ancho a largo (stack de todas las ciudades en una sola columna)
df_largo = df.melt(
    id_vars=['MES','AÑO'],
    value_vars=cols_destinos,
    var_name='DESTINO_NUM',
    value_name='MUNICIPIO'
)

# Eliminar filas sin destino
df_largo = df_largo.dropna(subset=['MUNICIPIO'])

# Contar cuántas veces aparece cada ciudad en cada MES y AÑO
df_resultado = (
    df_largo
    .groupby(['MES','AÑO','MUNICIPIO'], as_index=False)
    .size()
    .rename(columns={'size':'VISITAS'})
)

# Limpiar el formato de año (remover comas) antes de convertir a entero
df_resultado['AÑO'] = df_resultado['AÑO'].astype(str).str.replace(',', '', regex=False).astype(int)

# Ordenar por año, mes y ciudad
df_resultado = df_resultado.sort_values(by=['AÑO','MES','MUNICIPIO']).reset_index(drop=True)

### Almacenamiento

In [15]:
# Guardar el dataset procesado
df_resultado.to_csv('./Datasets/Datos_de_entrenamiento.csv', index=False)